## PDF Pre-processing

In [1]:
pdf_path = './resources/solving_olympiad_geometry.pdf'
DEFAULT_MODEL = "meta-llama/Llama-3.2-1B-Instruct" 
# OPTIONAL_MODEL = "google/gemma-2-2b"

In [2]:
import PyPDF2
from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer

from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

In [3]:
def validate_pdf(file_path: str) -> bool:
    if not os.path.exists(file_path):
        print(f"Error: File not found at path: {file_path}")
        return False
    if not file_path.lower().endswith('.pdf'):
        print("Error: File is not a PDF")
        return False
    return True

Convert PDF to a `.txt` file. This would simply read and dump the contents of the file. We set the maximum characters to 100k. 

For people converting their favorite novels into a podcast, they will have to add extra logic of going outside the Llama models context length which is 128k tokens.

In [4]:
def extract_text_from_pdf(file_path: str, max_chars: int = 100000) -> Optional[str]:
    if not validate_pdf(file_path):
        return None
    
    try:
        with open(file_path, 'rb') as file:
            # Create PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)
            
            # Get total number of pages
            num_pages = len(pdf_reader.pages)
            print(f"Processing PDF with {num_pages} pages...")
            
            extracted_text = []
            total_chars = 0
            
            # Iterate through all pages
            for page_num in range(num_pages):
                # Extract text from page
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                # Check if adding this page's text would exceed the limit
                if total_chars + len(text) > max_chars:
                    # Only add text up to the limit
                    remaining_chars = max_chars - total_chars
                    extracted_text.append(text[:remaining_chars])
                    print(f"Reached {max_chars} character limit at page {page_num + 1}")
                    break
                
                extracted_text.append(text)
                total_chars += len(text)
                print(f"Processed page {page_num + 1}/{num_pages}")
            
            final_text = '\n'.join(extracted_text)
            print(f"\nExtraction complete! Total characters: {len(final_text)}")
            return final_text
            
    except PyPDF2.PdfReadError:
        print("Error: Invalid or corrupted PDF file")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return None


Helper function to grab meta info about our PDF

In [5]:
# Get PDF metadata
def get_pdf_metadata(file_path: str) -> Optional[dict]:
    if not validate_pdf(file_path):
        return None
    
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            metadata = {
                'num_pages': len(pdf_reader.pages),
                'metadata': pdf_reader.metadata
            }
            return metadata
    except Exception as e:
        print(f"Error extracting metadata: {str(e)}")
        return None

Finally, we can run our logic to extract the details from the file

In [6]:
# Extract metadata first
print("Extracting metadata...")
metadata = get_pdf_metadata(pdf_path)
if metadata:
    print("\nPDF Metadata:")
    print(f"Number of pages: {metadata['num_pages']}")
    print("Document info:")
    for key, value in metadata['metadata'].items():
        print(f"{key}: {value}")

# Extract text
print("\nExtracting text...")
extracted_text = extract_text_from_pdf(pdf_path)

# Display first 500 characters of extracted text as preview
if extracted_text:
    print("\nPreview of extracted text (first 500 characters):")
    print("-" * 50)
    print(extracted_text[:500])
    print("-" * 50)
    print(f"\nTotal characters extracted: {len(extracted_text)}")

# Optional: Save the extracted text to a file
if extracted_text:
    output_file = 'extracted_text.txt'
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(extracted_text)
    print(f"\nExtracted text has been saved to {output_file}")

Extracting metadata...

PDF Metadata:
Number of pages: 21
Document info:
/Author: Trieu H. Trinh
/CreationDate: D:20240126100646+05'30'
/Creator: Springer
/Keywords: 
/ModDate: D:20240126100754+05'30'
/Subject: Nature, doi:10.1038/s41586-023-06747-5
/Title: Solving olympiad geometry without human demonstrations

Extracting text...
Processing PDF with 21 pages...
Processed page 1/21
Processed page 2/21
Processed page 3/21
Processed page 4/21
Processed page 5/21
Processed page 6/21
Processed page 7/21
Processed page 8/21
Processed page 9/21
Processed page 10/21
Processed page 11/21
Processed page 12/21
Processed page 13/21
Processed page 14/21
Processed page 15/21
Processed page 16/21
Processed page 17/21
Processed page 18/21
Processed page 19/21
Processed page 20/21
Processed page 21/21

Extraction complete! Total characters: 75333

Preview of extracted text (first 500 characters):
--------------------------------------------------
476 | Nature | Vol 625 | 18 January 2024
ArticleSolving

### Llama Pre-Processing

Now let's proceed to justify our distaste for writing regex and use that as a justification for a LLM instead:

At this point, have a text file extracted from a PDF of a paper. Generally PDF extracts can be messy due to characters, formatting, Latex, Tables, etc. 

One way to handle this would be using regex, instead we can also prompt the feather light Llama models to clean up our text for us. 

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

SYS_PROMPT = """
You are a world class text pre-processor, here is the raw data from a PDF, please parse and return it in a way that is crispy and usable to send to a podcast writer.

The raw data is messed up with new lines, Latex math and you will see fluff that we can remove completely. Basically take away any details that you think might be useless in a podcast author's transcript.

Remember, the podcast could be on any topic whatsoever so the issues listed above are not exhaustive

Please be smart with what you remove and be creative ok?

Remember DO NOT START SUMMARIZING THIS, YOU ARE ONLY CLEANING UP THE TEXT AND RE-WRITING WHEN NEEDED

Be very smart and aggressive with removing details, you will get a running portion of the text and keep returning the processed text.

PLEASE DO NOT ADD MARKDOWN FORMATTING, STOP ADDING SPECIAL CHARACTERS THAT MARKDOWN CAPATILISATION ETC LIKES

ALWAYS start your response directly with processed text and NO ACKNOWLEDGEMENTS about my questions ok?
Here is the text:
"""

Instead of having the model process the entire file at once, as you noticed in the prompt-we will pass chunks of the file. 

One issue with passing chunks counted by characters is, we lose meaning of words so instead we chunk by words:

In [8]:
def create_word_bounded_chunks(text, target_chunk_size):
    """
    Split text into chunks at word boundaries close to the target chunk size.
    """
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        word_length = len(word) + 1  # +1 for the space
        if current_length + word_length > target_chunk_size and current_chunk:
            # Join the current chunk and add it to chunks
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = word_length
        else:
            current_chunk.append(word)
            current_length += word_length
    
    # Add the last chunk if it exists
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

Let's load in the model and start processing the text chunks

In [9]:
accelerator = Accelerator()
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map=device,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)

In [10]:
# Makes sure everything runs on CPU and doesn't get split on GPU for Gemma
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [11]:
def process_chunk(text_chunk, chunk_num):
    """Process a chunk of text and return both input and output for verification"""
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]
    
    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            temperature=0.7,
            top_p=0.9,
            max_new_tokens=512
        )
    
    processed_text = tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt):].strip()
    
    # Print chunk information for monitoring
    #print(f"\n{'='*40} Chunk {chunk_num} {'='*40}")
    print(f"INPUT TEXT:\n{text_chunk[:500]}...")  # Show first 500 chars of input
    print(f"\nPROCESSED TEXT:\n{processed_text[:500]}...")  # Show first 500 chars of output
    print(f"{'='*90}\n")
    
    return processed_text

In [12]:
INPUT_FILE = "./resources/extracted_text.txt"  # Replace with file path
CHUNK_SIZE = 1000  # Adjust chunk size if needed

chunks = create_word_bounded_chunks(extracted_text, CHUNK_SIZE) # fixed bug
num_chunks = len(chunks)

In [13]:
num_chunks

75

In [14]:
# Read the file
with open(INPUT_FILE, 'r', encoding='utf-8') as file:
    text = file.read()

# Calculate number of chunks
num_chunks = (len(text) + CHUNK_SIZE - 1) // CHUNK_SIZE

# Create output file name
output_file = f"clean_{os.path.basename(INPUT_FILE)}"

In [15]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)
processed_text = "" #Initialize processed_text
with open(output_file, 'w', encoding='utf-8') as out_file:
    for chunk_num, chunk in enumerate(tqdm(chunks, desc="Processing chunks")):
        # Process chunk and append to complete text
        processed_chunk = process_chunk(chunk, chunk_num)
        processed_text += processed_chunk + "\n"
        
        # Write chunk immediately to file
        out_file.write(processed_chunk + "\n")
        out_file.flush()

Processing chunks:   0%|          | 0/75 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
476 | Nature | Vol 625 | 18 January 2024 ArticleSolving olympiad geometry without human demonstrations Trieu H. Trinh1,2 ✉, Yuhuai Wu1, Quoc V. Le1, He He2 & Thang Luong1 ✉ Proving mathematical theorems at the olympiad level represents a notable milestone in human-level automated reasoning1–4, owing to their reputed difficulty among the world’s best talents in pre-university mathematics. Current machine-learning approaches, however, are not applicable to most mathematical domains owing to the hi...

PROCESSED TEXT:
ng, owing to the difficulty among world's best talents in pre-university mathematics Current machine-learning approaches are not applicable to most mathematical domains due to high cost of translating human proofs into machine-verifiable format The problem is worse for geometry due to unique translation challenges resulting in scarcity of training data We propose AlphaGeometry a theorem prover for Euclidean plane geometry that synthesizes millions of theorems and

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
scratch on our large-scale synthetic data, to guide a symbolic deduction engine through infinite branching points in challenging problems. On a test set of 30 latest olympiad-level problems, AlphaGeometry solves 25, outperforming the previous best method that only solves ten problems and approaching the performance of an average International Mathematical Olympiad (IMO) gold medallist. Notably, AlphaGeometry produces human-readable proofs, solves all geometry problems in the IMO 2000 and 2015 un...

PROCESSED TEXT:
anching points in challenging problems. On a test set of 30 latest olympiad-level problems, AlphaGeometry solves 25, outperforming the previous best method that only solves ten problems and approaching the performance of an average International Mathematical Olympiad (IMO) gold medallist. Notably, AlphaGeometry produces human-readable proofs, solves geometry problems in the IMO 2000 and 2015 under human expert evaluation and discovers a generalized version of a t

...

INPUT TEXT:
factor of 3. Extended Data Fig. 5 | Human proof for the IMO 2008 P6. This is an unsolved problem by AlphaGeometry and also the hardest one among all 30 problems, with an average human score of only 0.28/7. This human proof uses four auxiliary constructions (diameters of circles W1 and W2) and high-level theorems such as the Pitot theorem and the notion of homothety. These high-level concepts are not available to our current version of the symbolic deduction engine both during synthetic data gene...

PROCESSED TEXT:
phaGeometry and the hardest one among all 30 problems, with an average human score of only 0.28/7 This human proof uses four auxiliary constructions (diameters of circles W1 and W2) and high-level theorems such as the Pitot theorem and the notion of homothety These high-level concepts are not available to our current version of the symbolic deduction engine both during synthetic data generation and proof search Supplying AlphaGeometry with the auxiliary construct

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
test-time performance of AlphaGeometry. Article Extended Data Fig. 6 | Analysis of AlphaGeometry performance under changes made to its training and testing. a, The effect of reducing training data on AlphaGeometry performance. At 20% of training data, AlphaGeometry still solves 21 problems, outperforming all other baselines. b , Evaluation on a larger set of 231 geometry problems, covering a diverse range of sources outside IMO competitions. The rankings of different machine solvers stays the sa...

PROCESSED TEXT:
ance under changes made to its training and testing
a, The effect of reducing training data on AlphaGeometry performance
At 20% of training data, AlphaGeometry still solves 21 problems, outperforming all other baselines
b, Evaluation on a larger set of 231 geometry problems, covering a diverse range of sources outside IMO competitions
The rankings of different machine solvers stays the same as in Table 1, with AlphaGeometry solving almost all problems
c, The effe

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
construct the random premises These actions include constructions to create new points that are related to others in a certain way, for example, collinear, incentre/excentre etc., and constructions that take a number as its parameter. ArticleExtended Data Table 2 | Three examples of algebraic reasoning (AR) in geometry theorem proving, with AR proof steps between the two tags <AR></AR> In AlphaGeometry, the engine AR can execute all three examples efficiently, under a unified procedure of Gaussi...

PROCESSED TEXT:
Arithmetic Reasoning in Geometry Theorems**
----------------------------------------

Arithmetic reasoning plays a crucial role in geometry, enabling us to derive important theorems from basic properties. In this section, we will explore three examples of algebraic reasoning in geometry theorems.

**Algebraic Reasoning in Geometric Theorems**
--------------------------------------------

Algebraic reasoning is a fundamental aspect of geometry, allowing us to mani

Let's print out the final processed versions to make sure things look good

In [16]:
print(f"\nProcessing complete!")
print(f"Input file: {INPUT_FILE}")
print(f"Output file: {output_file}")
print(f"Total chunks processed: {num_chunks}")

# Preview the beginning and end of the complete processed text
print("\nPreview of final processed text:")
print("\nBEGINNING:")
print(processed_text[:1000])
print("\n...\n\nEND:")
print(processed_text[-1000:])


Processing complete!
Input file: ./resources/extracted_text.txt
Output file: clean_extracted_text.txt
Total chunks processed: 101

Preview of final processed text:

BEGINNING:
ng, owing to the difficulty among world's best talents in pre-university mathematics Current machine-learning approaches are not applicable to most mathematical domains due to high cost of translating human proofs into machine-verifiable format The problem is worse for geometry due to unique translation challenges resulting in scarcity of training data We propose AlphaGeometry a theorem prover for Euclidean plane geometry that synthesizes millions of theorems and proofs across different levels of complexity
anching points in challenging problems. On a test set of 30 latest olympiad-level problems, AlphaGeometry solves 25, outperforming the previous best method that only solves ten problems and approaching the performance of an average International Mathematical Olympiad (IMO) gold medallist. Notably, AlphaGeomet